In [5]:
from abc import ABC, abstractmethod


class Expression(ABC):  # an interface
    

    @abstractmethod
    def reduce(self, to : str): # -> Money
        pass
    

class Money(Expression):    # implement the interface

    _amount = None     # In Java, protected
    _currency = None   # In Java, protected


    def __init__(self, amount, currency):
        self._amount = amount
        self._currency = currency
    
    
    def __eq__(self, object):
        # In Java, casting to Money first, then:
        return self._amount == object._amount and self.currency() == object.currency()


    @staticmethod
    def dollar(amount): # -> Money
        return Money(amount, "USD")
    

    @staticmethod
    def franc(amount):  # -> Money
        return Money(amount, "CHF")
    

    def times(self, multiplier):
        return Money(self._amount * multiplier, self._currency)


    def currency(self):
        return self._currency
    

    def __str__(self):
        return str(self._amount) + " " + self._currency
    

    def __add__(self, addend) -> Expression:
        return Sum(self, addend)
    

    def reduce(self, to : str): # -> Money
        return self


class Sum(Expression):      # implement interface

    augend: Money | None = None
    addend: Money | None = None


    def __init__(self, augend, addend):
        self.augend = augend
        self.addend = addend

    
    def reduce(self, to: str):
        amount : int = self.augend._amount + self.addend._amount
        return Money(amount, to)


class Bank:


    def reduce(self, source: Expression, to: str) -> Money:
        return source.reduce(to)

1. \\$5 + 10 CHF = \\$10 if rate is 2:1.
2. ~~\\$5 * 2 = \\$10~~.
3. ~~make `amount` private~~.
4. ~~Dollar side-effects?~~
5. Money rounding?
6. ~~`equals()`~~.
7. Equal null.
8. Equal object.
9. ~~5 CHF * 2 = 10 CHF~~.
10. ~~Dollar/Franc duplication~~.
11. ~~Common equals~~.
12. ~~Common times~~.
13. ~~Compare Francs with Dollars~~.
14. ~~Currency?~~
15. Delete testFrancMultiplication?
16. \\$5 + \\$5 = \\$10
17. Return `Money` from \\$5 + \\$5
18. ~~`Bank.reduce(Money)`~~
19. **Reduce `Money` with conversation**
20. `Reduce(Money, String)`

In [6]:
def testReduceMoneyDifferentCurrency():
    bank : Bank = Bank()
    bank.addRate("CHF", "USD", 2)
    result : Money = bank.reduce(Money.franc(2), "USD")
    assert Money.dollar(1) == result


testReduceMoneyDifferentCurrency()

AttributeError: 'Bank' object has no attribute 'addRate'

In [9]:
class Expression(ABC):  # an interface
    

    @abstractmethod
    def reduce(self, bank: Bank, to : str): # -> Money
        pass


class Money(Expression):    # implement the interface

    _amount = None     # In Java, protected
    _currency = None   # In Java, protected


    def __init__(self, amount, currency):
        self._amount = amount
        self._currency = currency
    
    
    def __eq__(self, object):
        # In Java, casting to Money first, then:
        return self._amount == object._amount and self.currency() == object.currency()


    @staticmethod
    def dollar(amount): # -> Money
        return Money(amount, "USD")
    

    @staticmethod
    def franc(amount):  # -> Money
        return Money(amount, "CHF")
    

    def times(self, multiplier):
        return Money(self._amount * multiplier, self._currency)


    def currency(self):
        return self._currency
    

    def __str__(self):
        return str(self._amount) + " " + self._currency
    

    def __add__(self, addend) -> Expression:
        return Sum(self, addend)
    

    def reduce(self, bank : Bank, to : str): # -> Money
        rate = bank.rate(self._currency, to)
        return Money(self._amount / rate, to)
    

class Sum(Expression):      # implement interface

    augend: Money | None = None
    addend: Money | None = None


    def __init__(self, augend, addend):
        self.augend = augend
        self.addend = addend

    
    def reduce(self, bank : Bank, to: str):
        amount : int = self.augend._amount + self.addend._amount
        return Money(amount, to)
    

class Bank:


    def reduce(self, source: Expression, to: str) -> Money:
        return source.reduce(self, to)
    

    def rate(self, from_: str, to: str) -> int:
        return 2 if from_ == "CHF" and to == "USD" else 1

That pesky rate of 2 still appears in both the test and the code.

To get rid of it, we
need to keep a table of rates in the `Bank` and look up a rate when we need it. We
could use a hashtable that maps pairs of currencies to rates.

In [11]:
class Pair:

    _from_ = None
    _to = None

    def __init__(self, from_, to):
        self._from_ = from_
        self._to = to
    
    def __eq__(self, object):
        return self._from_ == object._from_ and self._to == object._to
    
    def __hash__(self):
        return 0
    

class Bank:


    _rates : dict[Pair, int] = {}


    def reduce(self, source: Expression, to: str) -> Money:
        return source.reduce(self, to)
    

    def rate(self, from_: str, to: str) -> int:
        return self._rates[Pair(from_, to)]
    

    def addRate(self, from_: str, to: str, rate: int):
        self._rates[Pair(from_, to)] = rate


testReduceMoneyDifferentCurrency()

0 is a terrible hash value, but it has the advantage of being easy to implement, and it will get us running quickly.

Wait a minute! We got a red bar. If we ask for the rate from USD to USD, we expect the value to be 1. Because this was a surprise, let’s write a test:

In [12]:
def testIdentityRate():
    assert 1 == Bank().rate("USD", "USD")


testIdentityRate()

KeyError: <__main__.Pair object at 0x7ac765dfc4f0>

In [13]:
class Bank:


    _rates : dict[Pair, int] = {}


    def reduce(self, source: Expression, to: str) -> Money:
        return source.reduce(self, to)
    

    def rate(self, from_: str, to: str) -> int:
        if from_ == to:
            return 1
        return self._rates[Pair(from_, to)]
    

    def addRate(self, from_: str, to: str, rate: int):
        self._rates[Pair(from_, to)] = rate


testReduceMoneyDifferentCurrency()
testIdentityRate()

1. \\$5 + 10 CHF = \\$10 if rate is 2:1.
2. ~~\\$5 * 2 = \\$10~~.
3. ~~make `amount` private~~.
4. ~~Dollar side-effects?~~
5. Money rounding?
6. ~~`equals()`~~.
7. Equal null.
8. Equal object.
9. ~~5 CHF * 2 = 10 CHF~~.
10. ~~Dollar/Franc duplication~~.
11. ~~Common equals~~.
12. ~~Common times~~.
13. ~~Compare Francs with Dollars~~.
14. ~~Currency?~~
15. Delete testFrancMultiplication?
16. ~~\\$5 + \\$5 = \\$10~~
17. Return `Money` from \\$5 + \\$5
18. ~~`Bank.reduce(Money)`~~
19. ~~Reduce `Money` with conversation~~
20. ~~`Reduce(Money, String)`~~